# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [111]:
!pip install langchain

In [112]:
!pip install langchain-openai

In [113]:
import warnings
warnings.filterwarnings('ignore')

In [114]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [115]:
#!pip install pandas

In [116]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [117]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [118]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [119]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [120]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [121]:
# old code 
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [123]:
product = "Wireless headphones"
chain.invoke(product)

AIMessage(content='Introducing our wireless headphones - the perfect solution for untangling your life! With crystal clear sound quality and a comfortable fit, you can jam out to your favorite tunes without getting tied up in cords. Say goodbye to the hassle of constantly adjusting tangled wires and hello to the freedom of wireless listening. Get yours today and cut the cord on your old headphones!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 27, 'total_tokens': 98, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-54ea899f-f3da-4aaa-994c-c8d5795432f4-0', usage_metadata={'input_tokens': 27, 'output_tokens': 71, 'total_tokens': 98})

## SimpleSequentialChain

In [124]:
from langchain.chains import SimpleSequentialChain

In [125]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [126]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [127]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [128]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing our wireless headphones: the ultimate sound experience without the pesky tangled cords. Enjoy your favorite tunes with crystal-clear audio and the freedom to move around without being tied down. Say goodbye to untangling knots and hello to hassle-free listening. Get yours now and cut the cord!
I would rate this product a 9 out of 10. The wireless headphones offer crystal-clear audio and the freedom to move around without being restricted by tangled cords. The hassle-free listening experience provided by these headphones is a major plus. Overall, this product seems to offer a high-quality sound experience.

> Finished chain.


{'input': 'Wireless headphones',
 'output': 'I would rate this product a 9 out of 10. The wireless headphones offer crystal-clear audio and the freedom to move around without being restricted by tangled cords. The hassle-free listening experience provided by these headphones is a major plus. Overall, this product seems to offer a high-quality sound experience.'}

**Repeat the above twice for different products**

## SequentialChain

In [129]:
from langchain.chains import SequentialChain

In [130]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into Spanish. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='es_review' #Give a name to your output
                    )


In [131]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {es_review}, summarize this review in less than 10 words in Spanish.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='es_short_review' #give a name to this output
                    )


In [132]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {es_short_review} into Greek"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='gr_short_review'
                      )


In [133]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {es_short_review} into {gr_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [134]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['es_short_review','gr_short_review', 'rating'],
    verbose=True
)

In [135]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'es_short_review': 'Posible lote viejo o falsificación, sabor mediocre.',
 'gr_short_review': 'Πιθανό παλιό παρτίδα ή πλαστικό, μέτρια γεύση.',
 'rating': 'I would rate the quality of the translation as a 8 out of 10. The translation accurately conveys the meaning of the original phrase, although some nuances may have been lost.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [136]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [137]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [138]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
import json

In [139]:
llm = ChatOpenAI(temperature=0)

In [140]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [141]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [142]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [159]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

import json

# Define a function that cleans and parses the JSON output
def parse_output(x):
    content = x.content.strip()
    try:
        # Clean the JSON output if it starts and ends with markdown-style JSON formatting
        if content.startswith("```json") and content.endswith("```"):
            content = content[7:-3].strip()  # Remove the ```json and ``` parts
        # Parse the JSON content into a dictionary
        return json.loads(content)
    except json.JSONDecodeError:
        print("Failed to decode JSON:", content)
        # Provide a default fallback response to avoid crashes
        return {"destination": "DEFAULT", "next_inputs": content}

# Update the router chain to use the parsing function
router_chain = router_prompt | llm | RunnableLambda(parse_output)

# Define the final chain that uses the cleaned JSON output
chain = (
    RunnablePassthrough()
    | {
        "input": RunnablePassthrough(),
        "parsed_output": router_chain,  # Renamed for clarity
    }
    | RunnableLambda(lambda x: prompt_templates[x["parsed_output"]["destination"]].format(input=x["parsed_output"]["next_inputs"]))
    | llm
)



In [160]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")


In [161]:
response

AIMessage(content='Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to reduce the dimensionality of a dataset while preserving as much variance as possible. In linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset.\n\nThe eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector. By selecting the eigenvectors with the highest eigenvalues, we can project the data onto a lower-dimensional subspace that captures the most important information.\n\nIn simpler terms, PCA helps us identify the most important features in a dataset and reduce the complexity of the data while retaining as much useful information as possible. This can be particularly useful for visualizing high-dimensional data or for improving the performance of machine learning algorithms by reducing the number of input features.', ad

In [162]:
chain.invoke("What is black body radiation?")

AIMessage(content="Thank you for the compliment! Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which absorbs all incident radiation and emits radiation at all wavelengths. This radiation follows a specific distribution known as Planck's law, which describes how the intensity of radiation emitted by a black body varies with wavelength and temperature. Black body radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 63, 'total_tokens': 148, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b7de0541-9c11-4eec-96ff-eba3b6ff4cbe-0', usage_metadata

In [163]:
chain.invoke("what is 2 + 2")

AIMessage(content='Thank you for the compliment! The answer to 2 + 2 is 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 74, 'total_tokens': 92, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-572bb966-ea96-49e7-af01-a9c6ffc13d49-0', usage_metadata={'input_tokens': 74, 'output_tokens': 18, 'total_tokens': 92})

In [173]:
response = chain.invoke("What numbers are divisible by 3 and 5?")
print(response)


content='A number is divisible by both 3 and 5 if it is divisible by their least common multiple, which is 15. Therefore, the numbers that are divisible by both 3 and 5 are multiples of 15. Some examples of numbers that are divisible by 3 and 5 are 15, 30, 45, 60, 75, and so on.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 78, 'total_tokens': 157, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5d53ba09-72cb-4e0a-b428-2874ca522071-0' usage_metadata={'input_tokens': 78, 'output_tokens': 79, 'total_tokens': 157}


**Repeat the above at least once for different inputs and chains executions - Be creative!**